In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import load_model
from keras.initializers import glorot_normal, Zeros, Ones
import keras.backend as K
from keras.optimizers import RMSprop
import tensorflow as tf

Using TensorFlow backend.


In [2]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ubm1nc3_
  Stored in directory: /tmp/pip-ephem-wheel-cache-jetdfiw_/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [3]:
%%time
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from math import sqrt
from scipy import stats
from scipy.stats import norm, skew #for some statistics

CPU times: user 300 ms, sys: 80 ms, total: 380 ms
Wall time: 407 ms


In [4]:
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
train = pd.read_csv('../input/innoplexus/train_3pirksi/train.csv')
test = pd.read_csv('../input/innoplexus/test_xev14ad/test.csv')
subm = pd.read_csv('../input/innoplexus/sample_submission_usrypcc/sample_submission.csv')

In [8]:
train = train.fillna(method="ffill")
test = test.fillna(method="ffill")

In [9]:
train.head()

,id,Doc_ID,Sent_ID,Word,tag
0,1,1,1,Obesity,O
1,2,1,1,in,O
2,3,1,1,Low-,O
3,4,1,1,and,O
4,5,1,1,Middle-Income,O


In [10]:
words = list(set(train["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

184506

In [11]:
tags = list(set(train["tag"].values))
n_tags = len(tags); n_tags

3

In [12]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [13]:
getter = SentenceGetter(train)
sentences = getter.sentences
len(sentences)

191282

In [14]:
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [15]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [16]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)

In [17]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]

In [18]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [19]:
X[0]
y[0]

array([184502, 116761, 182119,  31898, 117943,  26521, 172679,  70356,
       119645,  65130, 119645,  31898,  37017, 134375, 105930, 184505,
       184505, 184505, 184505, 184505, 184505, 184505, 184505, 184505,
       184505, 184505, 184505, 184505, 184505, 184505, 184505, 184505,
       184505, 184505, 184505, 184505, 184505, 184505, 184505, 184505,
       184505, 184505, 184505, 184505, 184505, 184505, 184505, 184505,
       184505, 184505], dtype=int32)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2], dtype=int32)

In [20]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [21]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [23]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=2,
                    validation_split=0.1, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 154937 samples, validate on 17216 samples
Epoch 1/2
154937/154937 [==============================] - 498s 3ms/step - loss: 0.0226 - crf_viterbi_accuracy: 0.9914 - val_loss: 0.0124 - val_crf_viterbi_accuracy: 0.9942
Epoch 2/2
 34016/154937 [=====>........................] - ETA: 6:15 - loss: 0.0114 - crf_viterbi_accuracy: 0.9943

In [24]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [25]:
test_pred = model.predict(X_te, verbose=1)

19129/19129 [==============================] - 13s 675us/step


In [26]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [27]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))
print(classification_report(test_labels, pred_labels))

F1-score: 64.1%
             precision    recall  f1-score   support

indications       0.74      0.57      0.64      5189

avg / total       0.74      0.57      0.64      5189



In [28]:
class SentenceGetterTest(object):
    
    def __init__(self, data):
        self.n_sent = 191283
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w) for w in s["Word"].values.tolist()]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [29]:
testgetter = SentenceGetterTest(test)
test_sentence = testgetter.sentences

In [30]:
x_test_sent  = pad_sequences(maxlen=max_len, sequences= [[word2idx.get(w, 0) for w in s] for s in test_sentence], padding="post", value=n_words - 1)

In [31]:
# words = []
predss=[]
for i in range(len(x_test_sent)):
    if i%10000 ==0:
        print(i)
    idxs = i
    p = model.predict(np.array([x_test_sent[idxs]]))
    p = np.argmax(p, axis=-1)
    #print(test_sentence[idxs])
#     print(tags[p[0]])
    for w, pred in zip(test_sentence[idxs], p[0]):
        #print(w, tags[pred])
#         words.append(w)
        predss.append(tags[pred])
#     break

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [32]:
len(words)
len(predss)
subm.shape

184506

2909818

(2994463, 3)

In [33]:
predss.extend(subm['tag'][len(predss):])

In [34]:
len(predss)
subm.shape

2994463

(2994463, 3)

In [35]:
subm['tag']=predss
subm.to_csv('submission_nn.csv',index=False)

In [36]:
# # import the modules we'll need
# from IPython.display import HTML
# import pandas as pd
# import numpy as np
# import base64

# # function that takes in a dataframe and creates a text link to  
# # download it (will only work for files < 2MB or so)
# def create_download_link(df, title = "Download CSV file", filename = "out.csv"):  
#     csv = df.to_csv(index=False)
#     b64 = base64.b64encode(csv.encode())
#     payload = b64.decode()
#     html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
#     html = html.format(payload=payload,title=title,filename=filename)
#     return HTML(html)

# # create a random sample dataframe
# df = pd.DataFrame(pred, columns=['tag'])

# # create a link to download the dataframe
# create_download_link(df)

In [37]:
subm['tag'].value_counts()

O                2840115
I-indications     129223
B-indications      25125
Name: tag, dtype: int64

ANAGO

In [38]:
!pip install git+https://www.github.com/Hironsan/anago.git

  Cloning https://www.github.com/Hironsan/anago.git to /tmp/pip-req-build-zcbfg336
  Stored in directory: /tmp/pip-ephem-wheel-cache-wlosfewn/wheels/23/13/ee/c961bc7c80736d808ed1fe0b763c304c1f94bc18b4775f2fe0
Successfully built anago


In [39]:
!pip install allennlp
# !pip install --ignore-installed allennlp

    100% |████████████████████████████████| 5.6MB 7.1MB/s 
    100% |████████████████████████████████| 12.6MB 2.8MB/s 
    100% |████████████████████████████████| 133kB 35.4MB/s 
    100% |████████████████████████████████| 51kB 16.6MB/s 
    100% |████████████████████████████████| 512kB 21.4MB/s 
    100% |████████████████████████████████| 184kB 36.3MB/s 
    100% |████████████████████████████████| 122kB 35.5MB/s 
    100% |████████████████████████████████| 4.5MB 6.6MB/s 
    100% |████████████████████████████████| 552kB 29.7MB/s 
    100% |████████████████████████████████| 1.5MB 18.7MB/s 
    100% |████████████████████████████████| 51kB 22.9MB/s 
    100% |████████████████████████████████| 51kB 21.8MB/s 
    100% |████████████████████████████████| 143kB 32.3MB/s 
    100% |████████████████████████████████| 61kB 22.6MB/s 
    100% |████████████████████████████████| 51kB 21.1MB/s 
    100% |████████████████████████████████| 7.5MB 4.6MB/s 
    100% |████████████████████████████████| 92kB

In [40]:
import numpy as np

from anago.utils import load_data_and_labels, load_glove, filter_embeddings
from anago.models import ELModel
from anago.preprocessing import ELMoTransformer
from anago.trainer import Trainer

p = ELMoTransformer()

ModuleNotFoundError: No module named 'allennlp'

In [41]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [42]:
getter = SentenceGetter(train)
sentences = getter.sentences

In [43]:
x_train = [[w[0] for w in s] for s in sentences]
y_train = [[w[1] for w in s] for s in sentences]

In [44]:
p.fit(x_train, y_train)

AttributeError: 'numpy.ndarray' object has no attribute 'fit'

In [45]:
print('Loading word embeddings...')
EMBEDDING_PATH = '../input/glove6b100dtxt/glove.6B.100d.txt'
embeddings = load_glove(EMBEDDING_PATH)
embeddings = filter_embeddings(embeddings, p._word_vocab.vocab, 100)

Loading word embeddings...


NameError: name 'load_glove' is not defined

In [46]:
model = ELModel(char_embedding_dim=25,
                    word_embedding_dim=100,
                    char_lstm_size=25,
                    word_lstm_size=100,
                    char_vocab_size=p.char_vocab_size,
                    word_vocab_size=p.word_vocab_size,
                    num_labels=p.label_size,
                    embeddings=embeddings,
                    dropout=0.5)
model, loss = model.build()

NameError: name 'ELModel' is not defined

In [47]:
trainer = Trainer(model, preprocessor=p)
model.compile(loss=loss, optimizer='adam')

NameError: name 'Trainer' is not defined

In [48]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=2018)

In [49]:
trainer.train(x_train, y_train, x_test, y_test)

NameError: name 'trainer' is not defined

In [50]:
print('Saving the model...')
model.save('weights.h5', 'params.json')

Saving the model...


In [51]:
from allennlp.data import Instance

ModuleNotFoundError: No module named 'allennlp'